<a href="https://colab.research.google.com/github/SchrodingersSangru/11860_project_qaoa/blob/master/Qubo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit

     |████████████████████████████████| 6.1 MB 3.5 MB/s 
     |████████████████████████████████| 17.9 MB 113 kB/s 
     |████████████████████████████████| 236 kB 51.9 MB/s 
     |████████████████████████████████| 207 kB 46.4 MB/s 
     |████████████████████████████████| 2.1 MB 54.5 MB/s 
     |████████████████████████████████| 1.4 MB 48.0 MB/s 
     |████████████████████████████████| 634 kB 52.0 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 49 kB 6.5 MB/s 
     |████████████████████████████████| 38.2 MB 25 kB/s 
     |████████████████████████████████| 943 kB 47.3 MB/s 
     |████████████████████████████████| 3.5 MB 62.0 MB/s 
     |████████████████████████████████| 6.3 MB 43.9 MB/s 
  Created wheel for qiskit: filename=qiskit-0.31.0-py3-none-any.whl size=11728 sha256=3f2a029cfcdb0b009f390a48e915583269dd5a294e01956c6fd4c4b6dca8d9f5
  Stored in directory: /root/.cache/pip/wheels/29/dd/b2/1fe1a9ac92aaf75b267d893ae27329ea229f292a29301

In [ ]:
from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram
import numpy as np


/usr/local/lib/python3.7/dist-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/usr/local/lib/python3.7/dist-packages/qiskit/optimization/__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('optimization', 'qiskit_optimization', 'qiskit-optimization')


In [ ]:
# create a QUBO
qubo = QuadraticProgram()
qubo.binary_var('x')
qubo.binary_var('y')
qubo.minimize(quadratic={('x', 'x'): -3, ('x', 'y'): 4, ('y', 'y'): -4})
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: [ - 6 x^2 + 8 x*y - 8 y^2 ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1

Binaries
 x y
End



In [ ]:
op, offset = qubo.to_ising()
print('offset: {}'.format(offset))
print('operator:')
print(op)

offset: -2.5
operator:
SummedOp([
  0.5 * IZ,
  ZZ,
  ZI
])


/usr/local/lib/python3.7/dist-packages/qiskit/aqua/operators/operator_base.py:46: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.operators', 'qiskit.opflow', 'qiskit-terra')


In [ ]:
qp=QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 3 x_0 - 4 x_1 + [ 8 x_0*x_1 ]/2
Subject To

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1

Binaries
 x_0 x_1
End



In [ ]:
algorithm_globals.random_seed = 10598
quantum_instance = QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                   seed_simulator=algorithm_globals.random_seed,
                                   seed_transpiler=algorithm_globals.random_seed)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])
exact_mes = NumPyMinimumEigensolver()

NameError: ignored

In [ ]:
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [ ]:
exact_result = exact.solve(qubo)
print(exact_result)

In [ ]:
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

In [ ]:
rqaoa = RecursiveMinimumEigenOptimizer(qaoa, min_num_vars=1, min_num_vars_optimizer=exact)

In [ ]:
rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result)

In [ ]:
import numpy as np

from qiskit import Aer
from qiskit.optimization.applications.ising import stable_set
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver, QAOA
from qiskit.aqua import aqua_globals
from qiskit.aqua import QuantumInstance
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms.optimizers import COBYLA
from qiskit.optimization.problems import QuadraticProgram


w = np.array([[-3., 2.],
 [2., -4.]])


qubitOp, offset = stable_set.get_operator(w)
print('Offset:', offset)
print('Ising Hamiltonian:')
print(qubitOp.print_details())

# mapping Ising Hamiltonian to Quadratic Program
qp = QuadraticProgram()
qp.from_ising(qubitOp, offset)
qp.to_docplex().prettyprint()

aqua_globals.random_seed = np.random.default_rng(123)
seed = 10598
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)
opt = COBYLA()
qaoa = QAOA(optimizer = opt,quantum_instance=quantum_instance, p = 3)

# create minimum eigen optimizer based on qaoa
qaoa_optimizer = MinimumEigenOptimizer(qaoa)

# solve quadratic program
result = qaoa_optimizer.solve(qp)
print(result)

In [ ]:
#Using Grovers Search

# example to solve a quadratic problem using a Quantum optimization algorithm

from qiskit.optimization import QuadraticProgram
from qiskit.optimization.algorithms import GroverOptimizer

from qiskit import Aer

from docplex.mp.model import Model

# construct problem with DOcplex
model = Model('docplex_model')
x, y = model.binary_var_list(2)
model.minimize(-3*x*x + 4*x*y -4*y*y)

# convert DOcplex model to Qiskit Quadratic Program
qp = QuadraticProgram()
qp.from_docplex(model)

# define a Quantum backend on which to run the optimization
# a Qiskit simulator in this case
backend = Aer.get_backend('statevector_simulator')

# use the Grover Adaptive Search (GAS) to solve the optimization problem
grover = GroverOptimizer(num_value_qubits=3, quantum_instance=backend)
result = grover.solve(qp)

# printing results: 
print(result)